In [1]:
%run BoxPlotFunctions.py

Welcome to JupyROOT 6.26/10


In [2]:
import matplotlib as mpl

def setup_pgf():
    pgf_with_latex = {
        "pgf.texsystem": "pdflatex",
        "text.usetex": True,
        "font.family": "serif",
        "font.serif": [],
        "font.sans-serif": [],
        "font.monospace": [],
        "axes.labelsize": 10,
        "font.size": 10,
        "legend.fontsize": 6,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10,
        "pgf.preamble": "\n".join([
            r"\usepackage[utf8]{inputenc}",
            r"\usepackage[T1]{fontenc}",
            r"\usepackage[detect-all]{siunitx}",
        ])
    }
    mpl.use("pgf")
    mpl.rcParams.update(pgf_with_latex)
 # Set the figure size

    # Set the figure size based on LaTeX geometry settings
    geometry_textwidth = 384  # pt
    pt_to_inch = 1.0 / 72.27  # Conversion factor from points to inches
    plot_width = geometry_textwidth * pt_to_inch *0.5
    
    mpl.rcParams['figure.figsize'] = (plot_width, plot_width*0.9)  # Adjust the height as needed (6 inches in this example)
def restore_defaults():
    mpl.rcParams.update(mpl.rcParamsDefault)
setup_pgf()

In [10]:
def update_event_tags(event_tags):
    new_event_tags = []
    current_event_tag = event_tags[0]
    new_event_tag = 1

    for tag in event_tags:
        if tag != current_event_tag:
            current_event_tag = tag
            new_event_tag += 1
        new_event_tags.append(new_event_tag)

    return new_event_tags


def count_photons_in_range(fevts, fZ, za, zb):
    # Create a dictionary to store the count for each tag
    count_dict = {}

    # Iterate over each tag and location
    for tag, z in zip(fevts, fZ):
        # Check if the location falls within the given range
        if za <= z <= zb:
            # Increment the count for the current tag
            count_dict[tag] = count_dict.get(tag, 0) + 1

    # Create a list of counts for each tag in the original order
    rst = [count_dict.get(tag, 0) for tag in fevts]

    return rst
def get_segment(vector1, vector2, value):
    segment = [v2 for v1, v2 in zip(vector1, vector2) if v1 == value]
    return segment
def add_one_within_range(vector, start, end):
    result = 0
    for i, value in enumerate(vector):
        if start <= value <= end:
            result+=1
    return result
def process_segments(fevts, fZ, za, zb):
    results=[]
    for tag in set(fevts):
        segment = get_segment(fevts, fZ, tag)
        modified_segment = add_one_within_range(segment, za, zb)
        results.append(modified_segment)
    return results


def add_one_within_rangexy(vector1, vector2, start1, end1, start2, end2):
    result = 0
    for value1, value2 in zip(vector1, vector2):
        if start1 <= value1 <= end1 and start2 <= value2 <= end2:
            result+=1
    return result

def process_segmentsxy(fevts, fZ, fX, za, zb, xa, xb):
    results = []
    for tag in set(fevts):
        segment = get_segment(fevts, fZ, tag)
        segment2 = get_segment(fevts, fX, tag)
        modified_segment = add_one_within_rangexy(segment, segment2, za, zb, xa, xb)
        results.append(modified_segment)
    return results

def filter_vectors(vec1, vec2, minval, maxval):
    filtered_vec1 = []
    filtered_vec2 = []

    for val1, val2 in zip(vec1, vec2):
        if minval <= val2 <= maxval:
            filtered_vec1.append(val1)
            filtered_vec2.append(val2)

    return filtered_vec1, filtered_vec2

def tols_data(folder: str, output_filename: str, branch: str, branchName5: str):
    # Define the variables
    treeName1 = "EndOfEvent"
    branchName1 = "fGlueL"
    branchName2 = "fResinL"
    branchName3 = "fDetXpos"
    branchName4 = "fDetYpos"
    #branchName5 = "fLOapprox"
 
    treeName2 = "Detected"
    branchName6 = "fX"
    branchName7 = "fEvent"

    # Read data from root file
    print("Reading X data")
    with contextlib.redirect_stdout(io.StringIO()):
        #data1 = readRootDataFromFolder(folder, treeName1, branchName1)
        #data2 = readRootDataFromFolder(folder, treeName1, branchName2)
        #data3 = readRootDataFromFolder(folder, treeName1, branchName3)
        lengths = readRootDataFromFolder(folder, treeName1, branch)
        fX = readRootDataFromFolder(folder, "Detected", "fX")
        fY = readRootDataFromFolder(folder, "Detected", "fY")
        fZ = readRootDataFromFolder(folder, "Detected", "fZ")
        fevts = readRootDataFromFolder(folder, "Detected", "fEvent",'int')
        edep = readRootDataFromFolder(folder, "EndOfEvent", "fEdep")
    fevtsf=update_event_tags(fevts)
    LC=process_segments(fevtsf, fX, -3.2, 0)
    
    LC_L=process_segmentsxy(fevts, fZ, fX, -50, 0, -3.2, 0)
    LC_R=process_segmentsxy(fevts, fZ, fX, 0, 50, -3.2, 0)

    LC_R,edepf=filter_vectors(LC_R, edep, 0.5, 3)
    LC_L,edepf=filter_vectors(LC_L, edep, 0.5, 3)
    LC,edepf=filter_vectors(LC, edep, 0.5, 3)
    lengthsf,edepf=filter_vectors(lengths, edep, 0.5, 3)
    
    LO = np.divide(LC, edepf)
    LO_R = np.divide(LC_R, edepf)
    LO_L = np.divide(LC_L, edepf)

    return LC,LC_L,LC_R,edepf,LO,LO_L,LO_R,lengthsf
def tols_datainp(folder: str, output_filename: str, branch: str, branchName5: str):
    # Define the variables
    treeName1 = "EndOfEvent"

    # Read data from root file
    print("Reading X data")
    with contextlib.redirect_stdout(io.StringIO()):
        #data1 = readRootDataFromFolder(folder, treeName1, branchName1)
        #data2 = readRootDataFromFolder(folder, treeName1, branchName2)
        #data3 = readRootDataFromFolder(folder, treeName1, branchName3)
        data1 = readRootDataFromFolder(folder, treeName1, branch)
        data2 = readRootDataFromFolder(folder, treeName1, branchName5)
        fEdep = readRootDataFromFolder(folder, treeName1, "fEdep")

    return data1,data2,fEdep
def box_plot_tols(data1,data2,fEdep,name='test.pdf', fit_type='linear', add_fit=True):
    yMin = 0
    yMax = 6000
    saveFolder = "Figures/2307/Tols/"
    treeName3 = "EndOfEvent"
    branchName4 = "fDdep"
    geometry_textwidth = 384  # pt
    pt_to_inch = 1.0 / 72.27  # Conversion factor from points to inches
    plot_width = geometry_textwidth * pt_to_inch *0.5
    fig, ax = plt.subplots(figsize=(plot_width, plot_width*0.9))
    xMax = max(data1)
    xMin = min(data1)
    #print(len(data1))
    #print(len(data2))
    #print("data1")
    #print(data1[0:10])
    #print("data2")
    #print(data2[0:10])
    n = int(np.ceil(np.sqrt(len(data1)))*0.75)  # Number of ranges
    #print("n ranges")
    #print(n)
    #print(n)
    range_size = (xMax-xMin) / n

    # Create a list to store the data within each range
    data_in_ranges = []
    box_positions = []
    box_widths = []

    # Iterate over each range and extract the data within it
    for i in range(n):
        range_min = xMin  + i * range_size 
        range_max = range_min + range_size 
        #print(range_min, range_max)
        values_in_range = []
        for j in range(len(data1)):
            if range_min <= data1[j] < range_max:
                if data2[j] > 10 and fEdep[j]>0.5 and fEdep[j]<4:
                    values_in_range.append((data2[j]))
        data_in_ranges.append(values_in_range)

        # Calculate the box position and width
        box_position = (range_min + range_max) / 2 
        box_width = 0.9 * (range_max - range_min) 
        box_positions.append(box_position)
        box_widths.append(box_width)

    #print(data_in_ranges)
    # Plot the box plot for each range
    boxplot = ax.boxplot(data_in_ranges, positions=box_positions, widths=box_widths, showfliers=False,
                         patch_artist=False,  # Set patch_artist to False
                         whiskerprops=dict(color='blue', linewidth=0.5), capprops=dict(color='blue', linewidth=0.5),
                         medianprops=dict(color='blue', linewidth=0.5), flierprops=dict(marker='o', markersize=3,
                                                                         markerfacecolor='black'), whis=1.5,boxprops=dict(color='blue', linewidth=0.5))
    


    # Set the x-axis limits
    ax.set_xlim(xMin, xMax)
    # Set the x-axis limits
    #ax.set_ylim(0, 2000)
    # Round the x-axis tick labels to the first decimal place
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.2f}'))
        # Add a grid
    ax.grid(True, axis='y', linestyle='--', color='lightgray')
    # Add labels
    ax.set_xlabel(r"Length $\left(\si{\milli\meter}\right)$")
    ax.set_ylabel(r"Light Output $\left(\si{\gamma_d/\mega\electronvolt}\right)$")

    # Customize the color of all boxes
    for box in boxplot['boxes']:
        box.set_color('blue')

    # Set the number of divisions on the x-axis
    num_divisions = 6  # Choose the desired number of divisions
    ax.xaxis.set_major_locator(mticker.MaxNLocator(num_divisions))
    filtered_list = [list(filter(lambda x: not (x == float('inf') or x == float('-inf')), sublist)) for sublist in data_in_ranges]

    # Calculate the mean of each box
    box_means = [np.mean(data) for data in filtered_list]
    #ax.plot(box_positions, box_means, 'ro', markersize=5)
    #print("box_means")
    #print(box_means)
    
    # Plot the average as a red dot
    ax.plot(box_positions, box_means, 'ro', markersize=1,label='Mean', linestyle='None')

    
    # Remove the title
    ax.set_title('')


#ax.set_xlabel(r"Light Output $\left(\si{\sum\gamma_d/\mega\electronvolt}\right)$")  # Set x-axis label with units

    # Increase the font size of the tick labels


    # Calculate the fit line based on the fit type if add_fit is True
    #print(box_means)
    #print(box_positions)
    if add_fit:
        if fit_type == 'linear':
            print("linear")
            fit_coefficients = np.polyfit(box_positions, box_means, deg=1)
            fit_line = np.polyval(fit_coefficients, box_positions)
            r_squared = np.corrcoef(box_means, fit_line)[0, 1] ** 2
            fit_label = f'Fit: {fit_coefficients[0]:.1f}x + {fit_coefficients[1]:.1f}\n$R^2$: {r_squared:.2f}'
        elif fit_type == 'quadratic':
            print("quad")
            fit_coefficients = np.polyfit(box_positions, box_means, deg=2)
            fit_line = np.polyval(fit_coefficients, box_positions)
            r_squared = np.corrcoef(box_means, fit_line)[0, 1] ** 2
            fit_label = f'Fit: {fit_coefficients[0]:.1f}x^2 + {fit_coefficients[1]:.1f}x + {fit_coefficients[2]:.1f}\n$R^2$: {r_squared:.2f}'
        # Plot the fit line
        ax.plot(box_positions, fit_line, color=(0.25,0,0), linestyle='--',label=fit_label,linewidth=0.5)
        #print(fit_line)


    # Add the legend to the plot
    ax.legend()
    # Save the figure as a PDF
    plt.savefig(saveFolder+name, format='pdf', bbox_inches='tight')

    # Show the plot
    plt.show()
    return
    

In [11]:
# import matplotlib.pyplot as plt
import numpy as np
folder="../TierIIData/2023_07/IncrResinV2"
#folder="../TierIIData/2023_02/Rst_GC3_Muon_23-02-09v2"
fResinL,LC_L,fEdep=tols_datainp(folder, "Incr_Resin.pdf","fResinL","fHits")
fResinL = [value * 2 + 0.25 for value in fResinL]  # Convert to list and apply the transformation
LC_L=np.divide(LC_L,fEdep)/2
box_plot_tols(fResinL,LC_L,fEdep,'Incr_ResinV2.pdf')


Reading X data
linear


/tmp/ipykernel_13066/3456809203.py:255: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [12]:
# import matplotlib.pyplot as plt
import numpy as np
folder="../TierIIData/2023_07/Incr_Resin"
#folder="../TierIIData/2023_02/Rst_GC3_Muon_23-02-09v2"
fResinL,LC_L,fEdep=tols_datainp(folder, "Incr_Resin.pdf","fResinL","fHits")
fResinL = [value * 2 + 0.25 for value in fResinL]  # Convert to list and apply the transformation
LC_L=np.divide(LC_L,fEdep)/2
box_plot_tols(fResinL,LC_L,fEdep,'Incr_Resin.pdf')


Reading X data
linear


/tmp/ipykernel_13066/3456809203.py:255: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


# import matplotlib.pyplot as plt
import numpy as np
folder="../TierIIData/2023_07/Incr_Resin"
#folder="../TierIIData/2023_02/Rst_GC3_Muon_23-02-09v2"
fResinL,LC_L,fEdep=tols_datainp(folder, "Incr_Resin.pdf","fResinL","fHits")
fResinL = [value * 2 + 0.25 for value in fResinL]  # Convert to list and apply the transformation
LC_L=np.divide(LC_L,fEdep)/2
box_plot_tols(fResinL,LC_L,fEdep,'Incr_Resin.pdf')


In [13]:
folder="../TierIIData/2023_07/IncrGlueV2"
#folder="../TierIIData/2023_02/Rst_GC3_Muon_23-02-09v2"
fGlueL,LC_L,fEdep=tols_datainp(folder, "abctest.pdf","fGlueL","fHits")
LC_L=np.divide(LC_L,fEdep)/2
fGlueL = [value * 2 + 0.0 for value in fGlueL]  # Convert to list and apply the transformation
print(LC_L[0:10])
print(fGlueL[0:10])
box_plot_tols(fGlueL,LC_L,fEdep,'Incr_GlueV2.pdf')

Reading X data
[2317.08539639 2298.95013763 2265.95775178 2302.52601703 2293.0803735
 2301.44095625 2294.67346112 2311.15091525 2282.40542032 2299.64989962]
[0.179, 0.179, 0.179, 0.179, 0.179, 0.179, 0.179, 0.179, 0.179, 0.179]
linear


/tmp/ipykernel_13066/3456809203.py:255: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [14]:
folder="../TierIIData/2023_07/Incr_Glue"
#folder="../TierIIData/2023_02/Rst_GC3_Muon_23-02-09v2"
fGlueL,LC_L,fEdep=tols_datainp(folder, "abctest.pdf","fGlueL","fHits")
LC_L=np.divide(LC_L,fEdep)/2
fGlueL = [value * 2 + 0.0 for value in fGlueL]  # Convert to list and apply the transformation
print(LC_L[0:10])
print(fGlueL[0:10])
box_plot_tols(fGlueL,LC_L,fEdep,'Incr_Glue.pdf')

Reading X data
[2317.02744598 2348.15997296 2346.59091494 2324.65632002 2356.65726266
 2345.27308486 2358.26696785 2315.98805542 2350.54331438 2352.89092689]
[0.149, 0.149, 0.149, 0.149, 0.149, 0.149, 0.149, 0.149, 0.149, 0.149]
linear


/tmp/ipykernel_13066/3456809203.py:255: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [8]:
folder="../TierIIData/2023_07/Incr_DET_Y"
#folder="../TierIIData/2023_02/Rst_GC3_Muon_23-02-09v2"
fDetYpos,LC_L,fEdep=tols_datainp(folder, "abctest.pdf","fDetYpos","fphR")
LC_L=np.divide(LC_L,fEdep)
print(LC_L[0:10])
print(fDetYpos[0:10])
box_plot_tols(fDetYpos,LC_L,fEdep,'Incr_DetYPos.pdf','quadratic')

Reading X data
[2252.00573377 2248.44254373 2247.5005554  2249.58795967 2211.98057128
 2186.07534865 2227.80014003 2259.7199574  2229.05817832 2207.74274777]
[0.182, 0.182, 0.182, 0.182, 0.182, 0.182, 0.182, 0.182, 0.182, 0.182]
quad


/tmp/ipykernel_13066/633856238.py:255: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [9]:
folder="../TierIIData/2023_07/Incr_DET_X"
#folder="../TierIIData/2023_02/Rst_GC3_Muon_23-02-09v2"
fDetXpos,LC_L,fEdep=tols_datainp(folder, "abctest.pdf","fDetXpos","fphR")
LC_L=np.divide(LC_L,fEdep)
print(LC_L[0:10])
print(fDetXpos[0:10])
box_plot_tols(fDetXpos,LC_L,fEdep,'Incr_DetXPos.pdf','quadratic')

Reading X data
[2305.15354554 2275.4500142  2313.95743545 2304.09005802 2246.73171157
 2306.75456996 2291.28024786 2315.40474338 2318.4103571  2308.21821113]
[0.177, 0.177, 0.177, 0.177, 0.177, 0.177, 0.177, 0.177, 0.177, 0.177]
quad


/tmp/ipykernel_13066/633856238.py:255: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
